In [1]:
import sys
import numpy as np
sys.path.append('../')
from deepod.models.time_series import COUTA
from load_data import load_data
from deepod.metrics import ts_metrics
from deepod.metrics import point_adjustment 

In [2]:
train_imu,train_audio,train_spec,val_imu,val_audio,val_spec,val_imu_normal,val_audio_normal,val_spec_normal = load_data()

train_combine = np.concatenate([np.mean(train_audio,axis=-1),train_imu],axis=1)

test_audio_all = np.concatenate([val_audio_normal,val_audio],axis=0)
test_imu_all   = np.concatenate([val_imu_normal,val_imu],axis=0)
test_combine = np.concatenate([np.mean(test_audio_all,axis=-1),test_imu_all],axis=1)

labels_all = np.concatenate([np.zeros(val_audio_normal.shape[0]),np.ones(val_audio.shape[0])])

In [3]:
clf_imu = COUTA(epochs=50)
clf_imu.fit(train_imu)
clf_audio = COUTA(epochs=50)
clf_audio.fit(np.mean(train_audio,axis=-1))
clf_all = COUTA(epochs=50)
clf_all.fit(train_combine)

/home/iot/anaconda3/envs/ai/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


|>>> epoch: 01  |   loss: 0.155728, loss_oc: 0.042430, val_loss: 0.031455
|>>> epoch: 02  |   loss: 0.133675, loss_oc: 0.026026, val_loss: 0.021586
|>>> epoch: 03  |   loss: 0.121237, loss_oc: 0.018636, val_loss: 0.015936
|>>> epoch: 04  |   loss: 0.110298, loss_oc: 0.013062, val_loss: 0.010226
|>>> epoch: 05  |   loss: 0.102546, loss_oc: 0.008482, val_loss: 0.006869
|>>> epoch: 06  |   loss: 0.097316, loss_oc: 0.005577, val_loss: 0.004292
|>>> epoch: 07  |   loss: 0.091610, loss_oc: 0.003333, val_loss: 0.002603
|>>> epoch: 08  |   loss: 0.087255, loss_oc: 0.002056, val_loss: 0.001698
|>>> epoch: 09  |   loss: 0.083267, loss_oc: 0.001349, val_loss: 0.001079
|>>> epoch: 10  |   loss: 0.077365, loss_oc: 0.001490, val_loss: 0.001699
|>>> epoch: 11  |   loss: 0.067959, loss_oc: 0.001642, val_loss: 0.001726
|>>> epoch: 12  |   loss: 0.054251, loss_oc: 0.001825, val_loss: 0.001317
|>>> epoch: 13  |   loss: 0.041077, loss_oc: 0.000734, val_loss: 0.000351
|>>> epoch: 14  |   loss: 0.035226, lo

: 

In [ ]:
scores_audio      = clf_audio.decision_function(np.mean(test_audio_all,axis=-1))
scores_imu        = clf_imu.decision_function(test_imu_all)
scores_all        = clf_all.decision_function(test_combine)

In [ ]:
#auc, mean precision, F1, percision, recall
eval_metrics_audio = ts_metrics(labels_all, scores_audio)
eval_metrics_imu = ts_metrics(labels_all, scores_imu)
result_audio = eval_metrics_audio
result_imu = eval_metrics_imu
eval_metrics_all = ts_metrics(labels_all, scores_all)
result_all = eval_metrics_all

print(result_audio)
print(result_imu)
print(result_all)

(0.9201270685041234, 0.9254275646267124, 0.8444394502791934, 0.8166189111747851, 0.8742331288343558)
(0.7021627132220422, 0.669649183029929, 0.7403007963560592, 0.6, 0.9662576687116564)
(0.8351477307895361, 0.826606815398993, 0.7654937258837397, 0.6826923076923077, 0.8711656441717791)
